In [2]:
import numpy as np
import pickle
import dgl

Using backend: pytorch


In [3]:
with open("data/conceptnet/embedding_keys.pkl", "rb") as f:
    embedding_keys = pickle.load(f)

In [7]:
CORPUS = []
tokens = []
for i,item in enumerate(embedding_keys):
    words = item.replace("_", " ")
    embedding_keys[i] = words
    words = words.split(" ")
    if type(words) == str:
        words = [words]
    tokens = tokens + words

In [6]:
import torch
from transformers import AutoModel, BertModel, AutoTokenizer, BertTokenizer
from tokenizers.implementations import base_tokenizer

In [17]:
tokenizer = BertTokenizer("data/conceptnet/embedding_keys.txt")
tokenizer("go to bed")

{'input_ids': [0, 4156, 1345, 542, 1], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [18]:
len(tokenizer.vocab)

4445

In [56]:
# Store the model we want to use
MODEL_NAME = "bert-base-cased"

# We need to create the model and tokenizer
encoder = AutoModel.from_pretrained(MODEL_NAME)
decoder = BertModel.from_pretrained(MODEL_NAME, is_decoder=True)

Some weights of BertModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.self.query.weight', 'bert.encoder.layer.1.crossattention.self.query.bias', 'bert.encoder.layer.1.crossattention.self.key.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.encoder.layer.1.crossattention.self.value.weight', 'bert.encoder.layer.1.crossatte

In [60]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# This code can be factored into one-line as follow
tokens_pt2 = tokenizer("This is an input example", return_tensors="pt")

for key, value in tokens_pt2.items():
    print("{}:\n\t{}".format(key, value))

outputs2, pooled2 = encoder(**tokens_pt2)

input_ids:
	tensor([[ 101, 1188, 1110, 1126, 7758, 1859,  102]])
token_type_ids:
	tensor([[0, 0, 0, 0, 0, 0, 0]])
attention_mask:
	tensor([[1, 1, 1, 1, 1, 1, 1]])


In [35]:
tokens_pt2 = tokenizer(embedding_keys, return_tensors="pt", padding=True)

for key, value in tokens_pt2.items():
    print("{}:\n\t{}".format(key, value))
    
outputs2, pooled2 = model(**tokens_pt2)

input_ids:
	tensor([[  101, 14874,  3317,  ...,     0,     0,     0],
        [  101,  3439,  4207,  ...,     0,     0,     0],
        [  101,  3119,  6806,  ...,     0,     0,     0],
        ...,
        [  101,  1301,  1106,  ...,     0,     0,     0],
        [  101,  9981,   102,  ...,     0,     0,     0],
        [  101,   187,  5053,  ...,     0,     0,     0]])
token_type_ids:
	tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
attention_mask:
	tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [70]:
decoder_output = decoder(input_ids, encoder_hidden_states = outputs2)
for item in decoder_output:
    print(item.shape)

torch.Size([1, 8, 768])
torch.Size([1, 768])


In [11]:
with open("embedding_keys.txt", "w") as f:
    for item in list(set(tokens)):
        f.write(item + "\n")